<a href="https://colab.research.google.com/github/krishnagorrepati/NLP/blob/main/NLP_END_Session_08_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Session 08-Assignment

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [ ]:
%%bash
#python -m spacy download en
python -m spacy download de
python -m spacy download fr

  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907057 sha256=f0ef7ec98580fa57d26b3ba0783a8c13a8ca0cb5e67523008bf235402920b71a
  Stored in directory: /tmp/pip-ephem-wheel-cache-f1j5y573/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.2.5-cp36-none-any.whl size=14727027 sha256=e044d136a9a4b73977aaa3647f1c98c38f0a1a942e2d3f7a1dc991c8db17ba00
  Stored in directory: /tmp/pip-ephem-wheel-cache-de4lf99o/wheels/46/1b/e6/29b020e3f9420a24c3f463343afe5136aaaf955dbc9e46dfc5
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now lo

In [ ]:
SEED=1234 # Initial state to generate random number this is alive till kernel

random.seed(SEED)
np.random.seed(SEED) #Used for RNG in other devices
torch.manual_seed(SEED) #Used for RNG of all devices
torch.cuda.manual_seed(SEED) # Sets the seed to generate randome number for current GPU
torch.backends.cudnn.deterministic=True #A bool that, if True, causes cuDNN to only use deterministic convolution algorithm


In [ ]:
spacy_de = spacy.load('de')
#spacy_en = spacy.load('en')
spacy_fr = spacy.load('fr')

In [ ]:
def tokenize_fr(text):
  return [tok.text for tok in spacy_fr.tokenizer(text)][::-1]

def tokenize_de(text):
  return [tok.text for tok in spacy_de.tokenizer(text)]

In [ ]:
SRC = Field(tokenize=tokenize_fr,
            init_token='<sos>',
            eos_token = '<sos>',
            lower = True)
TRG = Field(tokenize=tokenize_de,
            init_token='<sos>',
            eos_token='<eos>',
            lower=True)

In [ ]:
!git clone --recursive https://github.com/multi30k/dataset.git multi30k-dataset

Cloning into 'multi30k-dataset'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 296 (delta 7), reused 14 (delta 3), pack-reused 262
Receiving objects: 100% (296/296), 17.52 MiB | 30.25 MiB/s, done.
Resolving deltas: 100% (55/55), done.
Submodule 'scripts/subword-nmt' (https://github.com/rsennrich/subword-nmt.git) registered for path 'scripts/subword-nmt'
Cloning into '/content/multi30k-dataset/scripts/subword-nmt'...
remote: Enumerating objects: 576, done.        
remote: Total 576 (delta 0), reused 0 (delta 0), pack-reused 576        
Receiving objects: 100% (576/576), 233.30 KiB | 10.14 MiB/s, done.
Resolving deltas: 100% (349/349), done.
Submodule path 'scripts/subword-nmt': checked out '80b7c1449e2e26673fb0b5cae993fe2d0dc23846'


In [ ]:
cd /content/multi30k-dataset/data/task1/raw


/content/multi30k-dataset/data/task1/raw


In [ ]:
!gunzip *.gz

gzip: *.gz: No such file or directory


In [ ]:
!ls

multi30k	     test_2017_flickr.de  test_2017_mscoco.fr  train.cs  val.de
test_2016_flickr.cs  test_2017_flickr.en  test_2018_flickr.cs  train.de  val.en
test_2016_flickr.de  test_2017_flickr.fr  test_2018_flickr.de  train.en  val.fr
test_2016_flickr.en  test_2017_mscoco.de  test_2018_flickr.en  train.fr
test_2016_flickr.fr  test_2017_mscoco.en  test_2018_flickr.fr  val.cs


In [ ]:
cp train.fr val.fr multi30k/

In [ ]:
!ls multi30k/

mmt_task1_test2016.tar.gz  test2016.fr	train.fr	 val.en
test2016.de		   train.de	training.tar.gz  val.fr
test2016.en		   train.en	val.de		 validation.tar.gz


In [ ]:
train_data,valid_data,test_data = Multi30k.splits(exts=('.fr','.de'),fields=(SRC,TRG),root='/content/multi30k-dataset/data/task1/raw')
#train_data,valid_data,test_data = Multi30k.splits(exts=('.fr','.de'),fields=(SRC,TRG))

In [ ]:
#print(f"Number of elements in Train: {len(train_data.exampled)}")
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


In [ ]:
print(vars(train_data.examples[0]))

{'src': ['.', 'buissons', 'de', 'près', 'dehors', 'sont', 'blancs', 'hommes', 'jeunes', 'deux'], 'trg': ['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.']}


In [ ]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [ ]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (eb) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 6461
Unique tokens in target (eb) vocabulary: 7855


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
BATCH_SIZE =128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data,valid_data,test_data),
    batch_size = BATCH_SIZE,
    device = device)

In [ ]:
device 

device(type='cuda')

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        # Here you can use nn.GRU(emb_dim, hid_dim) or 
        # nn.RNN(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell


In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell


In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 3
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(6461, 256)
    (rnn): LSTM(256, 512, num_layers=3, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=3, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=7855, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [ ]:
w = torch.zeros(3, 5)
print(w)
nn.init.uniform_(w)
print(w)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
tensor([[0.9177, 0.0244, 0.9050, 0.1171, 0.9647],
        [0.1508, 0.7504, 0.2281, 0.9425, 0.7614],
        [0.9219, 0.5419, 0.1153, 0.1308, 0.8945]])


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 19,253,423 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        optimizer.zero_grad()
        output = model(src, trg)
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model,iterator, criterion):
  model.eval()
  epoch_loss = 0
  with torch.no_grad():
    for i,batch in enumerate(iterator):
      src = batch.src
      trg = batch.trg
      output = model(src,trg,0) #turn off teacher forcing

      output_dim = output.shape[-1]
      output = output[1:].view(-1,output_dim)
      trg = trg[1:].view(-1)

      loss = criterion(output, trg)
      epoch_loss += loss.item()

  return epoch_loss/len(iterator)


In [ ]:
def epoch_time(start_time,end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time/60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 52s
	Train Loss: 5.215 | Train PPL: 184.090
	 Val. Loss: 5.050 |  Val. PPL: 155.995
Epoch: 02 | Time: 0m 53s
	Train Loss: 4.669 | Train PPL: 106.594
	 Val. Loss: 4.804 |  Val. PPL: 121.992
Epoch: 03 | Time: 0m 53s
	Train Loss: 4.320 | Train PPL:  75.169
	 Val. Loss: 4.678 |  Val. PPL: 107.543
Epoch: 04 | Time: 0m 53s
	Train Loss: 4.083 | Train PPL:  59.338
	 Val. Loss: 4.594 |  Val. PPL:  98.863
Epoch: 05 | Time: 0m 53s
	Train Loss: 3.951 | Train PPL:  51.970
	 Val. Loss: 4.499 |  Val. PPL:  89.930
Epoch: 06 | Time: 0m 54s
	Train Loss: 3.839 | Train PPL:  46.459
	 Val. Loss: 4.428 |  Val. PPL:  83.773
Epoch: 07 | Time: 0m 53s
	Train Loss: 3.740 | Train PPL:  42.117
	 Val. Loss: 4.383 |  Val. PPL:  80.073
Epoch: 08 | Time: 0m 53s
	Train Loss: 3.625 | Train PPL:  37.514
	 Val. Loss: 4.304 |  Val. PPL:  74.016
Epoch: 09 | Time: 0m 53s
	Train Loss: 3.524 | Train PPL:  33.913
	 Val. Loss: 4.270 |  Val. PPL:  71.499
Epoch: 10 | Time: 0m 53s
	Train Loss: 3.427 | Train PPL

In [ ]:

model.load_state_dict(torch.load('tut1-model.pt'))
test_loss = evaluate(model, test_iterator,criterion)
print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')


| Test Loss: 4.147 | Test PPL:  63.268 |
